In [1]:
from pynq.overlays.base import BaseOverlay
import pynq.lib.rgbled as rgbled
import threading
import time
import random
base = BaseOverlay("base.bit")

In [2]:
leds = [base.leds[0], base.leds[1], base.leds[2], base.leds[3], rgbled.RGBLED(4)]
napping = [False, False, False, False, False]
forks = [threading.Lock(), threading.Lock(), threading.Lock(), threading.Lock(), threading.Lock()]
btns = base.btns_gpio

In [3]:
cond = True

def blink(t, d, n):
    global cond
    for i in range(t):
        if (n == 4):
            if (leds[n].read() == 2):
                leds[n].off()
            else:
                leds[n].on(0x2)
        else:
            leds[n].toggle()
        time.sleep(d)
    leds[n].off()

def philosopher_t(_l, _r, num):
    global cond
    while cond:
        if napping[num]:
            napping[num] = False
            print('Philosopher {} is napping.'.format(num))
            blink(25, 0.2, num)
        else:
            if ((_l.acquire(False) == True) and (_r.acquire(False) == True)):
                napping[num] = True
                print('Philosopher {} is eating.'.format(num))
                blink(50, 0.1, num)
            else:
                print('Philosopher {} is starving.'.format(num))
                if _l.acquire(False) == False:
                    _l.release()
                if _r.acquire(False) == False:
                    _r.release()
                time.sleep(5)
    print('Philosopher {} is done.'.format(num))
    
def get_btns_t():
    global cond
    while cond:
        time.sleep(0.01)
        if btns.read() != 0:
            cond = False
    for i in range(5):
        leds[i].off()

threads = []
for i in range(5):
    t = threading.Thread(target=philosopher_t, args=(forks[i], forks[(i+1)%5], i))
    threads.append(t)
    t.start()

t = threading.Thread(target=get_btns_t, args=())
threads.append(t)
t.start()

for t in threads:
    name = t.getName()
    t.join()
    print('{} joined'.format(name))

Philosopher 0 is eating.
Philosopher 1 is starving.
Philosopher 2 is starving.
Philosopher 3 is starving.
Philosopher 4 is starving.
Philosopher 1 is eating.
Philosopher 2 is starving.
Philosopher 0 is napping.
Philosopher 3 is starving.Philosopher 4 is starving.

Philosopher 2 is eating.
Philosopher 3 is starving.Philosopher 1 is napping.

Philosopher 0 is starving.Philosopher 4 is starving.

Philosopher 3 is eating.
Philosopher 2 is napping.
Philosopher 4 is starving.
Philosopher 1 is starving.
Philosopher 0 is eating.
Philosopher 4 is starving.
Philosopher 1 is starving.
Philosopher 2 is starving.
Philosopher 3 is napping.
Philosopher 0 is napping.
Philosopher 1 is eating.Philosopher 4 is eating.Philosopher 2 is starving.


Philosopher 3 is starving.
Philosopher 0 is starving.
Philosopher 2 is eating.
Philosopher 1 is napping.
Philosopher 4 is napping.Philosopher 3 is starving.Philosopher 0 is eating.


Philosopher 3 is starving.
Philosopher 1 is starving.Philosopher 2 is napping.



In [5]:
cond = True

def blink(t, d, n):
    global cond, napping
    for i in range(t):
        if (n == 4):
            if (leds[n].read() == 2):
                leds[n].off()
            else:
                leds[n].on(0x2)
        else:
            leds[n].toggle()
        time.sleep(d)
    leds[n].off()

def philosopher_t(_l, _r, num):
    global cond, napping
    eating_time = random.randint(25, 75);
    napping_time = random.randint(1, eating_time//2);
    while cond:
        if napping[num]:
            napping[num] = False
            print('Philosopher {} is napping.'.format(num))
            blink(napping_time, 0.2, num)
        else:
            if ((_l.acquire(False) == True) and (_r.acquire(False) == True)):
                napping[num] = True
                print('Philosopher {} is eating.'.format(num))
                blink(eating_time, 0.1, num)
            else:
                print('Philosopher {} is starving.'.format(num))
                if _l.acquire(False) == False:
                    _l.release()
                if _r.acquire(False) == False:
                    _r.release()
                time.sleep(eating_time * 0.1)
    print('Philosopher {} is done.'.format(num))
    
def get_btns_t():
    global cond, napping
    while cond:
        time.sleep(0.01)
        if btns.read() != 0:
            cond = False
    for i in range(5):
        leds[i].off()

threads = []
for i in range(5):
    t = threading.Thread(target=philosopher_t, args=(forks[i], forks[(i+1)%5], i))
    threads.append(t)
    t.start()

t = threading.Thread(target=get_btns_t, args=())
threads.append(t)
t.start()

for t in threads:
    name = t.getName()
    t.join()
    print('{} joined'.format(name))

Philosopher 0 is eating.
Philosopher 1 is starving.
Philosopher 2 is eating.
Philosopher 3 is starving.
Philosopher 4 is starving.
Philosopher 1 is starving.
Philosopher 0 is napping.
Philosopher 4 is eating.
Philosopher 0 is starving.
Philosopher 2 is napping.
Philosopher 1 is starving.
Philosopher 3 is starving.
Philosopher 0 is eating.
Philosopher 1 is starving.
Philosopher 4 is napping.
Philosopher 2 is eating.
Philosopher 1 is starving.
Philosopher 3 is starving.
Philosopher 0 is napping.
Philosopher 4 is starving.
Philosopher 0 is eating.
Philosopher 1 is starving.
Philosopher 2 is napping.
Philosopher 0 is napping.
Philosopher 4 is starving.
Philosopher 1 is starving.
Philosopher 3 is eating.
Philosopher 2 is starving.
Philosopher 0 is eating.
Philosopher 1 is starving.
Philosopher 4 is starving.
Philosopher 0 is napping.
Philosopher 1 is starving.
Philosopher 3 is napping.
Philosopher 2 is eating.
Philosopher 0 is eating.
Philosopher 4 is starving.
Philosopher 1 is starving.
Ph